In [7]:
import pandas as pd
import numpy as np
 
# Charger les données des fichiers CSV
data_btc = pd.read_csv('../bitcoin_historical_data_cleaned.csv')
data_sp500 = pd.read_csv('../sp500_historical_data_cleaned.csv')
data_gold = pd.read_csv('../gold_historical_data_cleaned.csv')

#Convertir en format numerique 
data_btc['Close'] = pd.to_numeric(data_btc['Close'], errors='coerce')
data_sp500['Close'] = pd.to_numeric(data_sp500['Close'], errors='coerce')
data_gold['Close'] = pd.to_numeric(data_gold['Close'], errors='coerce')

#Calcul des rendements quotidiens 
data_btc['Return'] = data_btc['Close'].pct_change().round(3)
data_sp500['Return'] = data_sp500['Close'].pct_change().round(3)
data_gold['Return'] = data_gold['Close'].pct_change().round(3)

## Calcul des rendements annuels
annual_return_btc = round(data_btc['Return'].mean() * 252,3)
annual_return_sp500 = round(data_sp500['Return'].mean() * 252,3)
annual_return_gold = round(data_gold['Return'].mean() * 252,3)

# Calcul de la volatilité
volatility_btc = round(data_btc['Return'].std() * (252**0.5),3)
volatility_sp500 = round(data_sp500['Return'].std() * (252**0.5),3)
volatility_gold = round(data_gold['Return'].std() * (252**0.5),3)

# Calcul de la volatilité quotidienne
volatility_daily_btc = round(data_btc['Return'].std(),3)
volatility_daily_sp500 = round(data_sp500['Return'].std(),3)
volatility_daily_gold = round(data_gold['Return'].std(),3)

# Calcul de la volatilité annuelle
volatility_annual_btc = round(volatility_daily_btc * np.sqrt(252),3)
volatility_annual_sp500 = round(volatility_daily_sp500 * np.sqrt(252),3)
volatility_annual_gold = round(volatility_daily_gold * np.sqrt(252),3)

# Calcul du Sharpe Ratio
risk_free_rate = 0.01
sharpe_ratio_btc = round((annual_return_btc - risk_free_rate) / volatility_annual_btc,3)
sharpe_ratio_sp500 =round( (annual_return_sp500 - risk_free_rate) / volatility_annual_sp500,3)
sharpe_ratio_gold = round((annual_return_gold - risk_free_rate) / volatility_annual_gold,3)


# Function to calculate drawdown percentage
def calculate_drawdown_pct(data):
    """Calculates the drawdown percentage for a given DataFrame."""
    peak = data['Close'].cummax()
    drawdown = (data['Close'] - peak) / peak
    return (drawdown * 100).round(3) # Convert to percentage


# Calculate and add 'drawdown_pct' column to each DataFrame
data_btc['drawdown_pct'] = calculate_drawdown_pct(data_btc)
data_sp500['drawdown_pct'] = calculate_drawdown_pct(data_sp500)
data_gold['drawdown_pct'] = calculate_drawdown_pct(data_gold)

# Calcul du Sortino Ratio (en supposant un taux sans risque de 0.01)
risk_free_rate = 0.01
downside_risk_btc = round(np.std(data_btc['Return'][data_btc['Return'] < 0]),3)
downside_risk_sp500 =round( np.std(data_sp500['Return'][data_sp500['Return'] < 0]) ,3)
downside_risk_gold = round(np.std(data_gold['Return'][data_gold['Return'] < 0]),3)

sortino_ratio_btc = round((annual_return_btc - risk_free_rate) / (downside_risk_btc * np.sqrt(252)),3)
sortino_ratio_sp500 = round((annual_return_sp500 - risk_free_rate) / (downside_risk_sp500 * np.sqrt(252)),3)
sortino_ratio_gold = round((annual_return_gold - risk_free_rate) / (downside_risk_gold * np.sqrt(252)),3)

# Calcul de l'Ulcer Index
ulcer_index_btc = round(np.sqrt(np.mean(data_btc['drawdown_pct'] ** 2)),3)
ulcer_index_sp500 = round(np.sqrt(np.mean(data_sp500['drawdown_pct'] ** 2)),3)
ulcer_index_gold = round(np.sqrt(np.mean(data_gold['drawdown_pct'] ** 2)),3)

# Créer des DataFrames pour enregistrer les résultats
results_btc = pd.DataFrame({
    'Rendement Quotidien': data_btc['Return'],
    'Volatilité Quotidienne': volatility_daily_btc,
    'Volatilité Annuelle': volatility_annual_btc,
    'Sharpe Ratio': sharpe_ratio_btc,
    'Sortino Ratio': sortino_ratio_btc,
    'Ulcer Index': ulcer_index_btc
})

results_sp500 = pd.DataFrame({
    'Rendement Quotidien': data_sp500['Return'],
    'Volatilité Quotidienne': volatility_daily_sp500,
    'Volatilité Annuelle': volatility_annual_sp500,
    'Sharpe Ratio': sharpe_ratio_sp500,
    'Sortino Ratio': sortino_ratio_sp500,
    'Ulcer Index': ulcer_index_sp500
})


results_gold = pd.DataFrame({
    'Rendement Quotidien': data_gold['Return'],
    'Volatilité Quotidienne': volatility_daily_gold,
    'Volatilité Annuelle': volatility_annual_gold,
    'Sharpe Ratio': sharpe_ratio_gold,
    'Sortino Ratio': sortino_ratio_gold,
    'Ulcer Index': ulcer_index_gold
})




# Enregistrer les résultats en CSV
results_btc.to_csv('../ath/results_btc.csv', index=False)
results_sp500.to_csv('../ath/results_sp500.csv', index=False)
results_gold.to_csv('../ath/results_gold.csv', index=False)